# Continuous Data

In [ ]:
%matplotlib inline
import warnings
from IPython.display import display

warnings.filterwarnings("ignore")
# tag: remove-cell applied

## Quickstart

To run a continuous dataset:

In [ ]:
import pybmds

# create a continuous dataset
dataset = pybmds.ContinuousDataset(
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7],
)

# create a session
session = pybmds.Session(dataset=dataset)
session.add_default_models()

# execute the session; recommend best-fitting model
session.execute_and_recommend()

# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

## Continuous datasets

Continuous datasets can defined using group level summary data or individual response data.

A continuous summary dataset requires a list of doses, mean responses, standard deviations, and the total number of subjects. All lists must be the same size, and there should be one item in each list for each dose-group. 

You can also add optional attributes, such as `name`, `dose_name`, `dose_units`, `response_name`, `response_units`, etc.

For example:

In [ ]:
dataset = pybmds.ContinuousDataset(
    name="Body Weight from ChemX Exposure",
    dose_name="Concentration",
    dose_units="ppm",
    response_units="kg",
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7],
)
dataset.plot(figsize=(6,4))

A continuous individual dataset requires the dose and response for each individual. Both lists must be the same size. Additional options can also be added such as the name or units:

In [ ]:
dataset2 = pybmds.ContinuousIndividualDataset(
    name="Effect Y from ChemX Exposure",
    dose_name="Dose",
    dose_units="mg/kg/d",
    response_units="g",    
    doses=[
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
            20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
            40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 
            80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 
        ],
    responses=[
        8.57, 8.71, 7.99, 9.27, 9.47, 10.06, 8.65, 9.13, 9.01, 8.35, 
        8.66, 8.45, 9.26, 8.76, 7.85, 9.01, 9.09, 9.57, 10.14, 10.49, 
        9.87, 10.26, 9.76, 10.24, 10.54, 10.72, 8.83, 10.02, 9.18, 10.87, 
        11.56, 10.59, 11.99, 11.59, 10.59, 10.52, 11.32, 12.0, 12.12, 12.64,
        10.96, 12.13, 12.77, 13.06, 13.16, 14.04, 14.01, 13.9, 12.99, 13.13,
    ],
)
dataset2.plot(figsize=(6,4))

## Single model fit

You can fit a specific model to the dataset and plot/print the results. The printed results will include the BMD, BMDL, BMDU, p-value, AIC, etc. 

For example, to fit the Hill model, run the following code and plot the results:

In [ ]:
from pybmds.models import continuous

model = continuous.Hill(dataset)
model.execute()
model.plot()

To view an output report:

In [ ]:
print(model.text())

You can change the range and initial value for any parameter in the model by following the same steps above.

### Change input settings

The default settings when running a continuous analysis are a BMR of 1 standard deviation from the BMD, a 95% confidence interval, and a normally distributed and constant variance between dose groups. 

If you fit a single model to your dataset, you can change these settings:

In [ ]:
model = continuous.Hill(dataset)
print(model.settings.tbl())
print(model.priors_tbl())

You can change individual model settings:


In [ ]:
model = continuous.Hill(
    dataset,
    settings={
        "bmr": 0.1,
        "bmr_type": pybmds.ContinuousRiskType.RelativeDeviation,
        "disttype": pybmds.ContinuousDistType.normal_ncv,
    },
)
model.settings.priors.update("k", max_value=3)
print(model.settings.tbl())
print(model.priors_tbl())

For the Polynomial model, degrees can be increased to a maximum of the lesser of `n`-1 or 8, where `n` is the number of dose groups.

In [ ]:
continuous.Polynomial(dataset, settings={"degree": 2})
continuous.Polynomial(dataset, settings={"degree": 3})

### Change parameter settings

To see a preview of the initial parameter settings:

In [ ]:
model = continuous.Hill(dataset)
print(model.settings.tbl())
print(model.priors_tbl())

You can also change the initial parameter settings shown above for any run of a single continuous model. Continuing with the Hill model example, you can set the power parameter `n` to be equal to 1. You do this by changing the initial, minimum, and maximum values of that parameter:

In [ ]:
model = continuous.Hill(dataset)
model.settings.priors.update("n", initial_value=1, min_value=1, max_value=1)
print(model.priors_tbl())

## Multiple model fit (sessions) and model recommendation

To run all the default models, save the results, and save the plot of the fit of the recommended model with the data:

In [ ]:
# create a continuous dataset
dataset = pybmds.ContinuousDataset(
    doses=[0, 25, 50, 75, 100],
    ns=[20, 20, 20, 20, 20],
    means=[6, 8, 13, 25, 30],
    stdevs=[4, 4.3, 3.8, 4.4, 3.7],
)

session = pybmds.Session(dataset=dataset)
session.add_default_models()

# execute the session
session.execute()

# recommend a best-fitting model
session.recommend()

# print recommended model and plot recommended model with dataset
model_index = session.recommender.results.recommended_model_index
if model_index:
    model = session.models[model_index]
    model.plot()

You can also plot all models:

In [ ]:
session.plot()

In [ ]:
session.plot(colorize=False)

To print a summary table of modeling results, create a custom function:

In [ ]:
import pandas as pd

def summary_table(session):
    data = []
    for model in session.models:
        data.append([
            model.name(), 
            model.results.bmdl, 
            model.results.bmd, 
            model.results.bmdu,             
            model.results.tests.p_values[1],
            model.results.tests.p_values[2],
            model.results.tests.p_values[3], 
            model.results.fit.aic
        ])
    
    df = pd.DataFrame(
        data=data, 
        columns=["Model", "BMDL", "BMD", "BMDU", "P-Value 2", "P-Value 3", "P-Value 4", "AIC"]
    )
    return df

summary_table(session)

### Change model settings

To change model settings to all default models added to a session:

In [ ]:
session = pybmds.Session(dataset=dataset)
session.add_default_models(
    settings={
        "disttype": pybmds.ContinuousDistType.normal_ncv,
        "bmr_type": pybmds.ContinuousRiskType.AbsoluteDeviation,
        "bmr": 2,
        "alpha": 0.1,
    }
)

Here, we changed the settings to assume normally distributed and non-constant variance between dose groups and a 90% confidence interval. We also changed the BMR type to be absolute deviation with a value of 2. You can use the default BMR for the given BMR type or specify a different value. You can also change the`disttype` to be `DistType.log_normal`. If a log-normal distribution is selected, only the Hill and Exponential models will be run.

### Run subset of models and select best fit

You can select a set of models and find the best fit, rather than using all of the default continuous models. 

For example, to model average on the Hill, Linear, and Exponential 3:

In [ ]:
session = pybmds.Session(dataset=dataset)
session.add_model(pybmds.Models.Linear)
session.add_model(pybmds.Models.ExponentialM3)
session.add_model(pybmds.Models.Hill)

session.execute()

### Model recommendation and selection

The `pybmds` package may recommend a best-fitting model based on a decision tree, but expert judgment may be required for model selection. To run model recommendation and view a recommended model, if one is recommended:

In [ ]:
session.recommend()

if session.recommended_model is not None:
    display(session.recommended_model.plot())
    print(session.recommended_model.text())

You can select a best-fitting model and output reports generated will indicate this selection.  This is a manual action. The example below selects the recommended model, but any model in the session could be selected.

In [ ]:
session.select(model=session.recommended_model, notes="Lowest AIC; recommended model")